In [1]:
from footium_api import GqlConnection
from footium_api.queries import get_next_fixtures
import pandas as pd
from datetime import datetime
from my_info import MyInfo




In [2]:
gql = GqlConnection()
my_info = MyInfo()
clubIds = my_info.get_club_ids()



In [3]:
fixtures = get_next_fixtures(gql, clubIds, 100)

if len(fixtures) == 0:
    print("No fixtures found")
else:

    fixtures["localTime"] = fixtures["realWorldTimestamp"].dt.tz_convert(None)
    local_tz = datetime.now().astimezone().tzinfo
    fixtures["localTime"] = fixtures["realWorldTimestamp"].dt.tz_convert(local_tz)


    # preitty print the time until kickoff
    def format_timedelta(td):
        total_seconds = int(td.total_seconds())
        time_string = ""
        if total_seconds < 0:
            total_seconds = abs(total_seconds)
            time_string += "ago "
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        if hours > 0:
            time_string += f"{hours}h, {minutes}m"
        elif minutes > 0:
            time_string += f"{minutes}m, {seconds}s"
        else:
            time_string += f"{seconds}s"
        return time_string


    now = pd.Timestamp.now(tz="UTC")
    fixtures["timeToKickOff"] = fixtures["realWorldTimestamp"] - now
    fixtures["timeToKickOff"] = fixtures["timeToKickOff"].apply(format_timedelta)

    # order by realWorldTimestamp ascending
    fixtures = fixtures.sort_values(by="realWorldTimestamp")

    print(fixtures.head())
    # walk thrrough each row in the DataFrame and print the row
    # for index, row in fixtures.iterrows():
    # print(row)
    print(fixtures.to_string(index=False))



No fixtures found


In [4]:

from typing import List, Optional


def custom_get_next_fixtures(
    gql: GqlConnection, club_ids: List[int], max_games: Optional[int] = None
):
    query = """
query NextFixtures($clubIds: [Int!]!, $take: Int, $skip: Int) {
    nextFixtures: fixtures(
    where: {
        AND: [
        { clubFixtures: { some: { clubId: { in: $clubIds } } } },
        { state: { notIn: ["BEFORE_KICKOFF", "LIVE"] } }
        ]
    },
    take: $take, skip: $skip,
    orderBy: { realWorldTimestamp: asc }
    ) {
    realWorldTimestamp
    clubFixtures {
        isHome
        club {
        id
        name
        }
    }
    tournament {
        name
        id
        clubTournaments {
        clubId
        position
        }
    }
    }
}
    """

    variables = {
        "clubIds": club_ids,
    }

    # response = gql.execute(gql.gql(query), variable_values=variables)
    # response = gql.send_query(query, variables)
    # fixtures = response['nextFixtures']
    max_games = max_games or len(club_ids) * 2
    fixtures = gql.send_paging_query(query, variables, take=max_games)

    # Process the response into a DataFrame
    fixtures_data = []
    for fixture in fixtures:
        for clubFixture in fixture["clubFixtures"]:
            if clubFixture["club"]["id"] in club_ids:
                fixtures_data.append(
                    {
                        # "realWorldTimestamp": fixture['realWorldTimestamp'],
                        "realWorldTimestamp": pd.to_datetime(
                            fixture["realWorldTimestamp"], unit="ms", utc=True
                        ),
                        "clubId": clubFixture["club"]["id"],
                        "clubName": clubFixture["club"]["name"],
                        "isHome": clubFixture["isHome"],
                        "tournamentName": fixture["tournament"]["name"],
                        "tournamentId": fixture["tournament"]["id"],
                        "position": next(
                            (
                                ct["position"]
                                for ct in fixture["tournament"]["clubTournaments"]
                                if ct["clubId"] == clubFixture["club"]["id"]
                            ),
                            None,
                        ),
                    }
                )

    fixtures_df = pd.DataFrame(fixtures_data)
    return fixtures_df



In [5]:
fixtures = custom_get_next_fixtures(gql, clubIds, 100)

In [6]:
fixtures.head()

,realWorldTimestamp,clubId,clubName,isHome,tournamentName,tournamentId,position
0,2024-05-16 22:00:00+00:00,1033,Wime Stanley,False,Division 8 - League 97,447,6
1,2024-05-17 22:00:00+00:00,1834,Alford,False,Division 8 - League 110,473,5
2,2024-04-26 18:00:00+00:00,85,Nordenhave,True,Division 3 - League 2,9,1
3,2024-05-01 08:00:00+00:00,521,Hareham,False,Division 8 - League 67,387,1
4,2024-05-04 14:00:00+00:00,2388,Louryhitwool Albion,True,Division 6 - League 26,113,7


In [7]:
if len(fixtures) == 0:
    print("No fixtures found")
else:

    fixtures["localTime"] = fixtures["realWorldTimestamp"].dt.tz_convert(None)
    local_tz = datetime.now().astimezone().tzinfo
    fixtures["localTime"] = fixtures["realWorldTimestamp"].dt.tz_convert(local_tz)


    # preitty print the time until kickoff
    def format_timedelta(td):
        total_seconds = int(td.total_seconds())
        time_string = ""
        if total_seconds < 0:
            total_seconds = abs(total_seconds)
            time_string += "ago "
        hours, remainder = divmod(total_seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        if hours > 0:
            time_string += f"{hours}h, {minutes}m"
        elif minutes > 0:
            time_string += f"{minutes}m, {seconds}s"
        else:
            time_string += f"{seconds}s"
        return time_string


    now = pd.Timestamp.now(tz="UTC")
    fixtures["timeToKickOff"] = fixtures["realWorldTimestamp"] - now
    fixtures["timeToKickOff"] = fixtures["timeToKickOff"].apply(format_timedelta)

    # order by realWorldTimestamp ascending
    fixtures = fixtures.sort_values(by="realWorldTimestamp")

    print(fixtures.head())
    # walk thrrough each row in the DataFrame and print the row
    # for index, row in fixtures.iterrows():
    # print(row)
    print(fixtures.to_string(index=False))



          realWorldTimestamp  clubId       clubName  isHome  \
61 2024-04-23 08:00:00+00:00    1831  Bolebrielbury    True   
76 2024-04-23 08:00:00+00:00    2879       Rocklant   False   
75 2024-04-23 08:00:00+00:00    1508  Rydowle Villa   False   
62 2024-04-23 08:00:00+00:00     523  Holsover Park   False   
73 2024-04-23 08:00:00+00:00     521        Hareham    True   

            tournamentName  tournamentId  position                 localTime  \
61  Division 8 - League 78           409         6 2024-04-23 01:00:00-07:00   
76  Division 8 - League 77           407         5 2024-04-23 01:00:00-07:00   
75  Division 8 - League 66           385         3 2024-04-23 01:00:00-07:00   
62  Division 8 - League 78           409         3 2024-04-23 01:00:00-07:00   
73  Division 8 - League 67           387         1 2024-04-23 01:00:00-07:00   

   timeToKickOff  
61  ago 804h, 4m  
76  ago 804h, 4m  
75  ago 804h, 4m  
62  ago 804h, 4m  
73  ago 804h, 4m  
       realWorldTimestamp 